In [1]:
import torch
from transformers import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel

import argparse
import logging
from tqdm import trange

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np


In [10]:
! pip3 install pytorch-pretrained-bert
! pip3 install spacy ftfy==4.4.3
! python3 -m spacy download en

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
/home/niranjan/.local/lib/python3.8/site-packages/cryptography/hazmat/backends/openssl/x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(
     |████████████████████████████████| 13.9 MB 10.5 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
import spur
class LinuxEnv:

    def __init__(self, host, user, password, port):

        self.host = host
        self.user = user
        self.password = password
        self.port = port
        self.client = None

    def connect(self):
        self.client = spur.SshShell(hostname=self.host, username=self.user, password=self.password, port=self.port)


    def send(self, command):
        try:
            result = self.client.run(command.split(" "))
            return result.output
        except:
            return "<ERROR>"



In [3]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [27]:
class GenNet(nn.Module):

    def __init__(self):
        super(GenNet, self).__init__()

        self.llm =  GPT2LMHeadModel.from_pretrained('gpt2')
        
    def forward(self, x, past = None):

        x = self.llm(x, past)

        return x

In [19]:
genModel = GenNet()

genModel.cuda()


GenNet(
  (llm): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(50257, 768)
      (wpe): Embedding(1024, 768)
      (h): ModuleList(
        (0): Block(
          (ln_1): BertLayerNorm()
          (attn): Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
          )
          (ln_2): BertLayerNorm()
          (mlp): MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
          )
        )
        (1): Block(
          (ln_1): BertLayerNorm()
          (attn): Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
          )
          (ln_2): BertLayerNorm()
          (mlp): MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
          )
        )
        (2): Block(
          (ln_1): BertLayerNorm()
          (attn): Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
          )
          (ln_2): BertLayerNorm()
          (mlp): MLP(
            (c_fc): Conv1D()
            

In [4]:
def top_k_logits(logits, k):
    """
    Masks everything but the k top entries as -infinity (1e10).
    Used to mask logits such that e^-infinity -> 0 won't contribute to the
    sum of the denominator.
    """
    if k == 0:
        return logits
    else:
        values = torch.topk(logits, k)[0]
        batch_mins = values[:, -1].view(-1, 1).expand_as(logits)
        return torch.where(logits < batch_mins, torch.ones_like(logits) * -1e10, logits)

In [5]:
def sample_sequence(model, length, start_token=None, batch_size=None, context=None, temperature=1, top_k=0, device='cpu', sample=True):
    if start_token is None:
        assert context is not None, 'Specify exactly one of start_token and context!'
        context = torch.tensor(context, device=device, dtype=torch.long).unsqueeze(0).repeat(batch_size, 1)
    else:
        assert context is None, 'Specify exactly one of start_token and context!'
        context = torch.full((batch_size, 1), start_token, device=device, dtype=torch.long)
    prev = context
    output = context
    past = None
    for i in trange(length):
        outputs= model(prev, past_key_values=past)
        logits, past = outputs.logits, outputs.past_key_values
        logits = logits[:, -1, :] / temperature
        logits = top_k_logits(logits, k=top_k)
        log_probs = F.softmax(logits, dim=-1)
        if sample:
            prev = torch.multinomial(log_probs, num_samples=1)
        else:
            _, prev = torch.topk(log_probs, k=1, dim=-1)
        output = torch.cat((output, prev), dim=1)
    return output

In [6]:
import json
f = open('bandit.json')
levels = json.load(f)


levels

[{'name': 'Level 0',
  'instructions': 'The password for the next level is stored in a file called readme located in the home directory. Use this password to log into bandit1 using SSH. Whenever you find a password for a level, use SSH (on port 2220) to log into that level and continue the game.',
  'commands': 'ls, cd, cat, file, du, find'}]

In [9]:
device = "cuda"
genModel = GPT2LMHeadModel.from_pretrained('gpt2')
genModel.load_state_dict(torch.load("linux-gpt.pt"))
genModel.to(device)
cur_level = levels[0]
raw_context = "ls"
# raw_context = cur_level['instructions']
env = LinuxEnv("bandit.labs.overthewire.org", "bandit0", "bandit0", 2220)
env.connect()
sequence_length = 10
for i in range(10):
    context_tokens = tokenizer.encode(raw_context)
    out = sample_sequence(
      model=genModel, length=sequence_length,
      context=context_tokens, batch_size=1, device=device
    )
    out = out[:, len(context_tokens):].tolist()[0]
    command = tokenizer.decode(out)
    result = env.send(command)
    print(f'. {command} : {result}')
    # raw_context += f'. {command} : {result}'

100%|██████████| 10/10 [00:00<00:00, 82.80it/s]

.  – Computer systems/hardware with damaged seams twice : <ERROR>



100%|██████████| 10/10 [00:00<00:00, 83.15it/s]

.  from the Counter-Influence campaign. Overall it : <ERROR>



 20%|██        | 2/10 [00:00<00:00, 12.17it/s]

...


100%|██████████| 10/10 [00:00<00:00, 82.29it/s]

.  are reasonable ways to ensure that the bitcoin users who : <ERROR>



100%|██████████| 10/10 [00:00<00:00, 77.88it/s]

.  way of looking at trends:

Absolute : <ERROR>


.  for drawing into late small sublinear depths \(\Delta : <ERROR>
